# Demonstrating Tensorflow Modin Interoperability
## All the examples in this section are taken/ adapted from https://www.tensorflow.org/tutorials/load_data/pandas_dataframe

In [1]:
import tensorflow as tf
import modin.pandas as pd
import pandas

In [2]:
SHUFFLE_BUFFER = 500
BATCH_SIZE = 2

csv_file = tf.keras.utils.get_file('heart.csv', 'https://storage.googleapis.com/download.tensorflow.org/data/heart.csv')

modin_df = pd.read_csv(csv_file)
modin_df.head()

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init(runtime_env={'env_vars': {'__MODIN_AUTOIMPORT_PANDAS__': '1'}})

2023-04-06 11:54:12,027	INFO worker.py:1553 -- Started a local Ray instance.


age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   63    1   1       145   233    1        2      150      0      2.3      3   
1   67    1   4       160   286    0        2      108      1      1.5      2   
2   67    1   4       120   229    0        2      129      1      2.6      2   
3   37    1   3       130   250    0        0      187      0      3.5      3   
4   41    0   2       130   204    0        2      172      0      1.4      1   

   ca        thal  target  
0   0       fixed       0  
1   3      normal       1  
2   2  reversible       0  
3   0      normal       0  
4   0      normal       0

In [3]:
target = modin_df.pop('target')

In [4]:
numeric_feature_names = ['age', 'thalach', 'trestbps',  'chol', 'oldpeak']
numeric_features = modin_df[numeric_feature_names]
numeric_features.head()

age  thalach  trestbps  chol  oldpeak
0   63      150       145   233      2.3
1   67      108       160   286      1.5
2   67      129       120   229      2.6
3   37      187       130   250      3.5
4   41      172       130   204      1.4

In [5]:
tf.convert_to_tensor(numeric_features)

2023-04-06 11:54:16.000875: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.Tensor: shape=(303, 5), dtype=float64, numpy=
array([[ 63. , 150. , 145. , 233. ,   2.3],
       [ 67. , 108. , 160. , 286. ,   1.5],
       [ 67. , 129. , 120. , 229. ,   2.6],
       ...,
       [ 65. , 127. , 135. , 254. ,   2.8],
       [ 48. , 150. , 130. , 256. ,   0. ],
       [ 63. , 154. , 150. , 407. ,   4. ]])>

In [6]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(numeric_features)

ValueError: Failed to find data adapter that can handle input: <class 'modin.pandas.dataframe.DataFrame'>, <class 'NoneType'>

## Replicating statsmodels workflow with pandas

In [7]:
SHUFFLE_BUFFER = 500
BATCH_SIZE = 2

csv_file = tf.keras.utils.get_file('heart.csv', 'https://storage.googleapis.com/download.tensorflow.org/data/heart.csv')

pandas_df = pandas.read_csv(csv_file)
pandas_df.head()

age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   63    1   1       145   233    1        2      150      0      2.3      3   
1   67    1   4       160   286    0        2      108      1      1.5      2   
2   67    1   4       120   229    0        2      129      1      2.6      2   
3   37    1   3       130   250    0        0      187      0      3.5      3   
4   41    0   2       130   204    0        2      172      0      1.4      1   

   ca        thal  target  
0   0       fixed       0  
1   3      normal       1  
2   2  reversible       0  
3   0      normal       0  
4   0      normal       0

In [8]:
target = pandas_df.pop('target')

In [9]:
numeric_feature_names = ['age', 'thalach', 'trestbps',  'chol', 'oldpeak']
numeric_features = pandas_df[numeric_feature_names]
numeric_features.head()

age  thalach  trestbps  chol  oldpeak
0   63      150       145   233      2.3
1   67      108       160   286      1.5
2   67      129       120   229      2.6
3   37      187       130   250      3.5
4   41      172       130   204      1.4

In [10]:
tf.convert_to_tensor(numeric_features)

<tf.Tensor: shape=(303, 5), dtype=float64, numpy=
array([[ 63. , 150. , 145. , 233. ,   2.3],
       [ 67. , 108. , 160. , 286. ,   1.5],
       [ 67. , 129. , 120. , 229. ,   2.6],
       ...,
       [ 65. , 127. , 135. , 254. ,   2.8],
       [ 48. , 150. , 130. , 256. ,   0. ],
       [ 63. , 154. , 150. , 407. ,   4. ]])>

In [11]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(numeric_features)